In [73]:
import re
import os
import numpy as np

#### Prelab

In [1]:
!mkdir -p data
!mkdir -p data/{dev,test,train}

In [87]:
with open("transcriptions.txt") as f_tran:
    lines = f_tran.read().splitlines()  
    transcriptions = dict([line.split("\t") for line in lines])

In [88]:
with open("lexicon.txt") as f_lex:
    lines = f_lex.read().splitlines()  
    # Convert to lowercase
    lines = [line.lower() for line in lines]
    # keep one of the duplicates (the one without any number)
    lines = [re.sub("\(\d\)", "", line) for line in lines]
    #create final lexicon
    lexicon = dict([line.split("\t") for line in lines])

In [89]:
dir_file_pairs = [
    ("train", "training.txt"),
    ("dev", "validation.txt"),
    ("test", "testing.txt"),
]

phonemes_all=set()
# Create necessary files
for (dir, file_name) in dir_file_pairs:
    # Create "utt_ids"
    !cp filesets/{file_name} data/{dir}/uttids ##no actual error

    with open(f"data/{dir}/uttids") as f_uttids:
        uttids = f_uttids.read().splitlines() #check that they are opening
        
        # Create utt2spk
        with open(f"data/{dir}/utt2spk", "w") as f_utt2spk:
            speakers = [uttid.split("_")[0] for uttid in uttids]
            for uttid, speaker in zip(uttids, speakers):
                print(f"{uttid} {speaker}", file=f_utt2spk)

        # Create wav.scp
        with open(f"data/{dir}/wav.scp", "w") as f_wav:
            for uttid in uttids:
                wavpath = os.path.realpath(f"wav/{uttid}.wav")
                print(f"{uttid} {wavpath}", file=f_wav)
        
        # Create 2 text files (original & phonemes)
        with open(f"data/{dir}/text_init", "w") as f_text_init, open(f"data/{dir}/text", "w") as f_text:
            for uttid in uttids:
                sentence_id = uttid.split("_")[1]
                sentence = transcriptions[sentence_id]

                # Original text to text_init
                print(f"{uttid} {sentence}", file=f_text_init)
                
                # Convert to lowercase
                sentence = sentence.lower()

                # Remove other
                sentence = re.sub("-", " ", sentence) #create gap when finding "-" symbol
                sentence = re.sub("[^a-z' ]", "", sentence)
                print(sentence)
                phonemes = [lexicon[word] for word in sentence.split()]
                for item in phonemes[0].split(" ")[1:]:
                    phonemes_all.add(item)
                print(f"{uttid} sil{''.join(phonemes)} sil")
                print(f"{uttid} sil{''.join(phonemes)} sil", file=f_text)
print(list(phonemes_all))
final=list(phonemes_all)
final2=final.sort()
print(final)

she is thinner than i am
f1_003 sil sh iy ih z th ih n er dh ae n ay ae m sil
bright sunshine shimmers on the ocean
f1_004 sil b r ay t s ah n sh ay n sh ih m er z aa n dh iy ow sh ah n sil
nothing is as offensive as innocence
f1_005 sil n ah th ih ng ih z ae z ah f eh n s ih v ae z ih n ah s ah n s sil
where were you while we were away
f1_007 sil w eh r w er y uw w ay l w iy w er ah w ey sil
are your grades higher or lower than nancy's
f1_008 sil aa r y ao r g r ey d z hh ay er ao r l ow er dh ae n n ae n s iy z sil
he will allow a rare lie
f1_009 sil hh iy w ih l ah l aw ah r eh r l ay sil
will robin wear a yellow lily
f1_010 sil w ih l r aa b ah n w eh r ah y eh l ow l ih l iy sil
before thursday's exam review every formula
f1_012 sil b ih f ao r th er z d iy z ih g z ae m r iy v y uw eh v er iy f ao r m y ah l ah sil
the museum hires musicians every evening
f1_013 sil dh iy m y uw z iy ah m hh ay er z m y uw z ih sh ah n z eh v er iy iy v n ih ng sil
carl lives in a lively home
f1_

#### 4.1

In [147]:
!ln -sf ../wsj/s5/{steps,utils} ./
!mkdir -p local
!ln -s steps/score_kaldi.sh local/score.sh
!mkdir -p conf
!cp mfcc.conf conf/
!mkdir -p data/lang data/local/dict data/local/lm_tmp data/local/nist_lm

ln: ./steps: cannot overwrite directory
ln: ./utils: cannot overwrite directory
ln: failed to create symbolic link 'local/score.sh': File exists


#### 4.2

In [126]:
!echo 'sil' > data/local/dict/silence_phones.txt
!echo 'sil' > data/local/dict/optional_silence.txt

In [127]:
phones=set()
#find non-silence phonemes and add them (sorted)
with open("lexicon.txt") as f_lexicon, open("data/local/dict/nonsilence_phones.txt", "w") as f_nonsil_phones:
    for line in f_lexicon.readlines():
        line_phones = line.split("\t")[1].strip().split()
        for phone in line_phones:
            phones.add(phone)
    
    phones.discard('sil')
    # Sort the remaining phones
    phones = sorted(list(phones))
    # Print one phone per line
    for i, phone in enumerate(phones):
        print(phone, file=f_nonsil_phones)

In [128]:
with open("data/local/dict/lexicon.txt", "w") as f_lexicon:
    temp_list = sorted((phones + ["sil"])) #dont forget to include the phoneme
    for i, phone in enumerate(temp_list):
        print(f"{phone} {phone}", file=f_lexicon)

In [129]:
# lm_train.text (+dev + test)
for dir in ["dev", "test", "train"]:
    with open(f"data/local/dict/lm_{dir}.text", "w") as f_lm, open(f"data/{dir}/text") as f_text:
        for line in f_text.readlines():
            # First token of every line is the uttid, the rest is the sentence
            uttid, *sentence = line.split()
            
            # Add <s> and </s>
            full_line = [uttid] + ["<s>"] + sentence + ["</s>"]
            print(" ".join(full_line), file=f_lm)

In [130]:
!touch data/local/dict/extra_questions.txt

In [131]:
!bash build-lm-all.sh

LOGFILE:/dev/null
$bin/ngt -i="$inpfile" -n=$order -gooout=y -o="$gzip -c > $tmpdir/ngram.${sdict}.gz" -fd="$tmpdir/$sdict" $dictionary $additional_parameters >> $logfile 2>&1
$scr/build-sublm.pl $verbose $prune $prune_thr_str $smoothing "$additional_smoothing_parameters" --size $order --ngrams "$gunzip -c $tmpdir/ngram.${sdict}.gz" -sublm $tmpdir/lm.$sdict $additional_parameters >> $logfile 2>&1
LOGFILE:/dev/null
$bin/ngt -i="$inpfile" -n=$order -gooout=y -o="$gzip -c > $tmpdir/ngram.${sdict}.gz" -fd="$tmpdir/$sdict" $dictionary $additional_parameters >> $logfile 2>&1
$scr/build-sublm.pl $verbose $prune $prune_thr_str $smoothing "$additional_smoothing_parameters" --size $order --ngrams "$gunzip -c $tmpdir/ngram.${sdict}.gz" -sublm $tmpdir/lm.$sdict $additional_parameters >> $logfile 2>&1
LOGFILE:/dev/null
$bin/ngt -i="$inpfile" -n=$order -gooout=y -o="$gzip -c > $tmpdir/ngram.${sdict}.gz" -fd="$tmpdir/$sdict" $dictionary $additional_parameters >> $logfile 2>&1
$bin/ngt -i="$inpfile" -

In [132]:
!bash compile-lm-all.sh

inpfile: data/local/lm_tmp/lm_phone_ug.ilm.gz
outfile: /dev/stdout
loading up to the LM level 1000 (if any)
dub: 10000000
OOV code is 1362
OOV code is 1362
Saving in txt format to /dev/stdout
inpfile: data/local/lm_tmp/lm_phone_bg.ilm.gz
outfile: /dev/stdout
loading up to the LM level 1000 (if any)
dub: 10000000
OOV code is 1362
OOV code is 1362
Saving in txt format to /dev/stdout
inpfile: data/local/lm_tmp/lm_train_ug.ilm.gz
outfile: /dev/stdout
loading up to the LM level 1000 (if any)
dub: 10000000
OOV code is 1362
OOV code is 1362
Saving in txt format to /dev/stdout
inpfile: data/local/lm_tmp/lm_train_bg.ilm.gz
outfile: /dev/stdout
loading up to the LM level 1000 (if any)
dub: 10000000
OOV code is 1362
OOV code is 1362
Saving in txt format to /dev/stdout
inpfile: data/local/lm_tmp/lm_test_ug.ilm.gz
outfile: /dev/stdout
loading up to the LM level 1000 (if any)
dub: 10000000
OOV code is 409
OOV code is 409
Saving in txt format to /dev/stdout
inpfile: data/local/lm_tmp/lm_test_bg.ilm.g

In [133]:
!bash prepare_lang_all.sh

/home/subdiduntil2/kaldi/egs/usc/utils/prepare_lang.sh data/local/dict <oov> data/local/lang data/lang
Checking data/local/dict/silence_phones.txt ...
--> reading data/local/dict/silence_phones.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> data/local/dict/silence_phones.txt is OK

Checking data/local/dict/optional_silence.txt ...
--> reading data/local/dict/optional_silence.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> data/local/dict/optional_silence.txt is OK

Checking data/local/dict/nonsilence_phones.txt ...
--> reading data/local/dict/nonsilence_phones.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> data/local/dict/nonsilence_phones.txt is OK

Checking disjoint: silence_phones.txt, nonsilence_phones.txt
--> disjoint property is OK.

Checking data/local/dict/lexicon.txt
--> reading data/local/dict/

In [134]:
# Sort feats.scp, text, utt2spk in data/{train,dev,test}
for dir in ["data/dev", "data/test", "data/train"]:
    for file_name in ["wav.scp", "text", "utt2spk"]:
        !sort {dir}/{file_name} > {dir}/{file_name}{"_temp"}
        !mv {dir}/{file_name}{"_temp"} {dir}/{file_name}
        
for dir in ["data/dev", "data/test", "data/train"]:
    !utils/utt2spk_to_spk2utt.pl {dir}/utt2spk > {dir}/spk2utt

In [135]:
!bash perplexity.sh

inpfile: data/local/lm_tmp/lm_phone_ug.ilm.gz
outfile: lm_phone_ug.ilm.blm
evalfile: data/local/dict/lm_test.text
loading up to the LM level 1000 (if any)
dub: 10000000
OOV code is 1362
OOV code is 1362
Start Eval
OOV code: 1362
%% Nw=13162 PP=54.56 PPwp=19.75 Nbo=0 Noov=367 OOV=2.79%
inpfile: data/local/lm_tmp/lm_phone_bg.ilm.gz
outfile: lm_phone_bg.ilm.blm
evalfile: data/local/dict/lm_test.text
loading up to the LM level 1000 (if any)
dub: 10000000
OOV code is 1362
OOV code is 1362
Start Eval
OOV code: 1362
%% Nw=13162 PP=27.65 PPwp=10.01 Nbo=606 Noov=367 OOV=2.79%
inpfile: data/local/lm_tmp/lm_phone_ug.ilm.gz
outfile: lm_phone_ug.ilm.blm
evalfile: data/local/dict/lm_dev.text
loading up to the LM level 1000 (if any)
dub: 10000000
OOV code is 1362
OOV code is 1362
Start Eval
OOV code: 1362
%% Nw=5078 PP=56.46 PPwp=21.16 Nbo=0 Noov=148 OOV=2.91%
inpfile: data/local/lm_tmp/lm_phone_bg.ilm.gz
outfile: lm_phone_bg.ilm.blm
evalfile: data/local/dict/lm_dev.text
loading up to the LM level 10

In [140]:
!bash timit_format_data.sh

Preparing train, dev and test data
Preparing language models for test
arpa2fst --disambig-symbol=#0 --read-symbol-table=data/lang_test/words.txt - data/lang_test/G_train_ug.fst 
LOG (arpa2fst[5.5.1071~1-19185]:Read():arpa-file-parser.cc:94) Reading \data\ section.
LOG (arpa2fst[5.5.1071~1-19185]:Read():arpa-file-parser.cc:149) Reading \1-grams: section.
WARNING (arpa2fst[5.5.1071~1-19185]:Read():arpa-file-parser.cc:219) line 7 [-4.39135	f1_003] skipped: word 'f1_003' not in symbol table
WARNING (arpa2fst[5.5.1071~1-19185]:Read():arpa-file-parser.cc:219) line 22 [-4.39135	f1_004] skipped: word 'f1_004' not in symbol table
WARNING (arpa2fst[5.5.1071~1-19185]:Read():arpa-file-parser.cc:219) line 30 [-4.39135	f1_005] skipped: word 'f1_005' not in symbol table
WARNING (arpa2fst[5.5.1071~1-19185]:Read():arpa-file-parser.cc:219) line 35 [-4.39135	f1_007] skipped: word 'f1_007' not in symbol table
WARNING (arpa2fst[5.5.1071~1-19185]:Read():arpa-file-parser.cc:219) line 41 [-4.39135	f1_008] ski

#### 4.3

In [137]:
!bash make_mfcc_all.sh

./conf/mfcc.conf: line 1: --use-energy=false: command not found
./conf/mfcc.conf: line 2: --sample-frequency=22050: command not found
./steps/make_mfcc.sh ./data/train
./steps/make_mfcc.sh: moving ./data/train/feats.scp to ./data/train/.backup
utils/validate_data_dir.sh: Successfully validated data-directory ./data/train
./steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
./steps/make_mfcc.sh: Succeeded creating MFCC features for train
./steps/make_mfcc.sh ./data/test
./steps/make_mfcc.sh: moving ./data/test/feats.scp to ./data/test/.backup
utils/validate_data_dir.sh: Successfully validated data-directory ./data/test
./steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
./steps/make_mfcc.sh: Succeeded creating MFCC features for test
./steps/make_mfcc.sh ./data/dev
./steps/make_mfcc.sh: moving ./data/dev/feats.scp to ./data/dev/.backup
utils/validate_data_dir.sh: Successfully validated data-directory ./dat

In [138]:
!bash mfcc_length.sh

feat-to-dim ark:data/train/data/raw_mfcc_train.1.ark - 
13
feat-to-len scp:data/train/feats.scp ark,t:data/train/feats.lengths 
f1_003 317 
f1_004 371 
f1_005 399 
f1_007 328 
f1_008 464 


#### 4.4

In [139]:
!bash steps/train_mono.sh data/train data/lang exp/mono

steps/train_mono.sh data/train data/lang exp/mono
steps/train_mono.sh: Initializing monophone system.
steps/train_mono.sh: Compiling training graphs
steps/train_mono.sh: Aligning data equally (pass 0)
steps/train_mono.sh: Pass 1
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 2
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 3
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 4
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 5
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 6
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 7
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 8
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 9
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 10
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 11
steps/train_mono.sh: Pass 12
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 13
steps/train_mono.sh: Pass 14
steps/train_mono.

In [141]:
!cp data/lang_test/G_train_ug.fst data/lang_test/G.fst
!bash utils/mkgraph.sh data/lang_test exp/mono exp/mono_graph_ug
!bash steps/decode.sh exp/mono_graph_ug data/dev exp/mono/decode_dev_ug
!bash steps/decode.sh exp/mono_graph_ug data/test exp/mono/decode_test_ug

tree-info exp/mono/tree 
tree-info exp/mono/tree 
fsttablecompose data/lang_test/L_disambig.fst data/lang_test/G.fst 
fstminimizeencoded 
fstpushspecial 
fstdeterminizestar --use-log=true 
fstisstochastic data/lang_test/tmp/LG.fst 
0.0422584 0.04188
[info]: LG not stochastic.
fstcomposecontext --context-size=1 --central-position=0 --read-disambig-syms=data/lang_test/phones/disambig.int --write-disambig-syms=data/lang_test/tmp/disambig_ilabels_1_0.int data/lang_test/tmp/ilabels_1_0.61993 data/lang_test/tmp/LG.fst 
fstisstochastic data/lang_test/tmp/CLG_1_0.fst 
0.0422584 0.04188
[info]: CLG not stochastic.
make-h-transducer --disambig-syms-out=exp/mono_graph_ug/disambig_tid.int --transition-scale=1.0 data/lang_test/tmp/ilabels_1_0 exp/mono/tree exp/mono/final.mdl 
fstrmsymbols exp/mono_graph_ug/disambig_tid.int 
fsttablecompose exp/mono_graph_ug/Ha.fst data/lang_test/tmp/CLG_1_0.fst 
fstdeterminizestar --use-log=true 
fstminimizeencoded 
fstrmepslocal 
fstisstochastic exp/mono_graph_ug/

In [142]:
!cp data/lang_test/G_train_bg.fst data/lang_test/G.fst
!bash utils/mkgraph.sh data/lang_test exp/mono exp/mono_graph_bg
!bash steps/decode.sh exp/mono_graph_bg data/dev exp/mono/decode_dev_bg
!bash steps/decode.sh exp/mono_graph_bg data/test exp/mono/decode_test_bg

tree-info exp/mono/tree 
tree-info exp/mono/tree 
fstpushspecial 
fstdeterminizestar --use-log=true 
fsttablecompose data/lang_test/L_disambig.fst data/lang_test/G.fst 
fstminimizeencoded 
fstisstochastic data/lang_test/tmp/LG.fst 
-0.00724196 -0.0078443
fstcomposecontext --context-size=1 --central-position=0 --read-disambig-syms=data/lang_test/phones/disambig.int --write-disambig-syms=data/lang_test/tmp/disambig_ilabels_1_0.int data/lang_test/tmp/ilabels_1_0.65633 data/lang_test/tmp/LG.fst 
fstisstochastic data/lang_test/tmp/CLG_1_0.fst 
-0.00724196 -0.0078443
make-h-transducer --disambig-syms-out=exp/mono_graph_bg/disambig_tid.int --transition-scale=1.0 data/lang_test/tmp/ilabels_1_0 exp/mono/tree exp/mono/final.mdl 
fstdeterminizestar --use-log=true 
fstrmsymbols exp/mono_graph_bg/disambig_tid.int 
fstminimizeencoded 
fsttablecompose exp/mono_graph_bg/Ha.fst data/lang_test/tmp/CLG_1_0.fst 
fstrmepslocal 
fstisstochastic exp/mono_graph_bg/HCLGa.fst 
0.000396412 -0.00892248
add-self-l

In [143]:
!bash steps/align_si.sh data/train data/lang exp/mono exp/mono_ali
!bash steps/train_deltas.sh 2000 10000 data/train data/lang exp/mono_ali exp/tri

steps/align_si.sh data/train data/lang exp/mono exp/mono_ali
steps/align_si.sh: feature type is delta
steps/align_si.sh: aligning data in data/train using model from exp/mono, putting alignments in exp/mono_ali
steps/diagnostic/analyze_alignments.sh --cmd run.pl data/lang exp/mono_ali
analyze_phone_length_stats.py: WARNING: optional-silence sil is seen only 31.0318664643% of the time at utterance begin.  This may not be optimal.
analyze_phone_length_stats.py: WARNING: optional-silence sil is seen only 27.5417298938% of the time at utterance end.  This may not be optimal.
steps/diagnostic/analyze_alignments.sh: see stats in exp/mono_ali/log/analyze_alignments.log
steps/align_si.sh: done aligning data.
steps/train_deltas.sh 2000 10000 data/train data/lang exp/mono_ali exp/tri
steps/train_deltas.sh: accumulating tree stats
steps/train_deltas.sh: getting questions for tree-building, via clustering
steps/train_deltas.sh: building the tree
WARNING (gmm-init-model[5.5.1071~1-19185]:InitAmGmm(

In [144]:
!cp data/lang_test/G_train_ug.fst data/lang_test/G.fst
!bash utils/mkgraph.sh data/lang_test exp/tri exp/tri_graph_ug
!bash steps/decode.sh exp/tri_graph_ug data/dev exp/tri/decode_dev_ug
!bash steps/decode.sh exp/tri_graph_ug data/test exp/tri/decode_test_ug

tree-info exp/tri/tree 
tree-info exp/tri/tree 
fsttablecompose data/lang_test/L_disambig.fst data/lang_test/G.fst 
fstdeterminizestar --use-log=true 
fstpushspecial 
fstminimizeencoded 
fstisstochastic data/lang_test/tmp/LG.fst 
0.0422584 0.04188
[info]: LG not stochastic.
fstcomposecontext --context-size=3 --central-position=1 --read-disambig-syms=data/lang_test/phones/disambig.int --write-disambig-syms=data/lang_test/tmp/disambig_ilabels_3_1.int data/lang_test/tmp/ilabels_3_1.73097 data/lang_test/tmp/LG.fst 
fstisstochastic data/lang_test/tmp/CLG_3_1.fst 
0.0422584 0
[info]: CLG not stochastic.
make-h-transducer --disambig-syms-out=exp/tri_graph_ug/disambig_tid.int --transition-scale=1.0 data/lang_test/tmp/ilabels_3_1 exp/tri/tree exp/tri/final.mdl 
fstrmepslocal 
fstrmsymbols exp/tri_graph_ug/disambig_tid.int 
fstdeterminizestar --use-log=true 
fsttablecompose exp/tri_graph_ug/Ha.fst data/lang_test/tmp/CLG_3_1.fst 
fstminimizeencoded 
fstisstochastic exp/tri_graph_ug/HCLGa.fst 
0.1

In [146]:
!cp data/lang_test/G_train_bg.fst data/lang_test/G.fst
!bash utils/mkgraph.sh data/lang_test exp/tri exp/tri_graph_bg
!bash steps/decode.sh exp/tri_graph_bg data/dev exp/tri/decode_dev_bg
!bash steps/decode.sh exp/tri_graph_bg data/test exp/tri/decode_test_bg

tree-info exp/tri/tree 
tree-info exp/tri/tree 
fsttablecompose data/lang_test/L_disambig.fst data/lang_test/G.fst 
fstminimizeencoded 
fstpushspecial 
fstdeterminizestar --use-log=true 
fstisstochastic data/lang_test/tmp/LG.fst 
-0.00724196 -0.0078443
fstcomposecontext --context-size=3 --central-position=1 --read-disambig-syms=data/lang_test/phones/disambig.int --write-disambig-syms=data/lang_test/tmp/disambig_ilabels_3_1.int data/lang_test/tmp/ilabels_3_1.79687 data/lang_test/tmp/LG.fst 
fstisstochastic data/lang_test/tmp/CLG_3_1.fst 
0 -0.00784433
make-h-transducer --disambig-syms-out=exp/tri_graph_bg/disambig_tid.int --transition-scale=1.0 data/lang_test/tmp/ilabels_3_1 exp/tri/tree exp/tri/final.mdl 
fsttablecompose exp/tri_graph_bg/Ha.fst data/lang_test/tmp/CLG_3_1.fst 
fstrmsymbols exp/tri_graph_bg/disambig_tid.int 
fstminimizeencoded 
fstrmepslocal 
fstdeterminizestar --use-log=true 
fstisstochastic exp/tri_graph_bg/HCLGa.fst 
0.000430787 -0.0186276
HCLGa is not stochastic
add-

#### 4.5

In [68]:
!pip install git+https://github.com/vesis84/kaldi-io-for-python

  Cloning https://github.com/vesis84/kaldi-io-for-python to /tmp/pip-req-build-ii5okauz
  Running command git clone -q https://github.com/vesis84/kaldi-io-for-python /tmp/pip-req-build-ii5okauz
  Created wheel for kaldi-io: filename=kaldi_io-0.9.5-py3-none-any.whl size=14117 sha256=1a3300e4496122033f2ff5162d4b55d53b3ed618c52c4772430d11462141716c
  Stored in directory: /tmp/pip-ephem-wheel-cache-pj7jrfxn/wheels/72/49/11/51883995a35bfb337190adb4b7e35d4d6a01d3873661e28812
Successfully built kaldi-io


In [69]:
from dnn.torch_dataset import *
!source ./path.sh
trainset = TorchSpeechDataset(recipe_dir='../wsj/s5', ali_dir='tri1_ali', feature_context=3, dset='train_1kshort')

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/subdiduntil2/.local/lib/python3.8/site-packages/kaldi_io/kaldi_io.py", line 97, in cleanup
    raise SubprocessFailed('cmd %s returned %d !' % (cmd,ret))
kaldi_io.kaldi_io.SubprocessFailed: cmd copy-feats scp:../wsj/s5/data/train_1kshort/feats.scp ark:- | apply-cmvn --utt2spk=ark:../wsj/s5/data/train_1kshort/utt2spk ark:../w

ValueError: need at least one array to concatenate